<a href="https://colab.research.google.com/github/nvthong2303/ChatRealTime-plus_Socket.IO/blob/master/machine_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tải xuống dataset cho quá trình huấn luyện mô hình

In [19]:
from tensorflow.keras.datasets import mnist
from matplotlib import pyplot as plt

# tải dữ liệu từ api keras
(trainX, trainY), (testX, testY) = mnist.load_data()

xử lý dữ liệu vừa tải xuống

In [21]:
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))

In [22]:
from tensorflow.keras.utils import to_categorical

trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [24]:
trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX = trainX / 255.0
testX = testX / 255.0

định nghĩa model

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))

	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

đánh giá mô hình

In [27]:
from sklearn.model_selection import KFold

def evaluate_model(dataX, dataY, n_folds=5):
    # prepare cross validation
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    # enumerate splits
    for train_ix, test_ix in kfold.split(dataX):
        # define model
        model = define_model()
        # select rows for train and test
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        # fit model
        history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
        # evaluate model
        _, acc = model.evaluate(testX, testY, verbose=0)
        print('> %.3f' % (acc * 100.0))
        # stores scores
        model.save('final_model.h5')
    return

evaluate_model(trainX, trainY);

> 98.600
> 98.608
> 98.650
> 98.850
> 98.783


đánh giá mô hình bằng bộ test từ mnist

In [29]:
from tensorflow.keras.models import load_model

def run_test_harness():
	# load model
	model = load_model('final_model.h5')
	# evaluate model on test dataset
	_, acc = model.evaluate(testX, testY, verbose=0)
	print('> %.3f' % (acc * 100.0))
 
# entry point, run the test harness
run_test_harness()

> 98.620


đưa ra dự đoán

1. dự đoán 1 bức ảnh đầu vào :

In [30]:
# make a prediction for a new image.
from numpy import argmax
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
 
# load and prepare the image
def load_image(filename):

	img = load_img(filename, grayscale=True, target_size=(28, 28))

	img = img_to_array(img)

	img = img.reshape(1, 28, 28, 1)

	img = img.astype('float32')
	img = img / 255.0
	return img
 
def run_example():
	img = load_image('sample_image.png')

	model = load_model('final_model.h5')

	predict_value = model.predict(img)
	digit = argmax(predict_value)
	print(digit)
 
run_example()

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


7
